## SECTION ONE
### DVDRENTAL

preparing the environment

In [ ]:
import getpass
import os

password = getpass.getpass()
command = 'sudo -S -i -u postgres pg_restore --dbname=dvdrental --create --verbose "/home/brian/Desktop/geopro/DATA ENGINEER INTERVIEW/dataset/SECTION ONE DATA SET/dvdrental.tar"'
os.system('echo %s | %s' % (password, command))

 ····


### Dumping sql result to json

In [67]:
import psycopg2

conn = psycopg2.connect(database="dvdrental", user = "postgres", password = "ABCabc123", host = "127.0.0.1", port = "5432")

In [68]:
import pandas.io.sql as psql

data = psql.read_sql('''
select json_agg(t) from ( select concat_ws(' ', first_name, last_name) as customer_name, email, address.address as address, 
json_agg((payment.customer_id, staff_id, rental_id)) as payment from customer 
join payment on payment.customer_id = customer.customer_id 
left join address on address.address_id = customer.address_id
group by first_name, last_name, email, address
)t ;''', conn)

In [63]:
data.json_agg[0][:5]

[{'customer_name': 'Aaron Selby',
  'email': 'aaron.selby@sakilacustomer.org',
  'address': '1519 Santiago de los Caballeros Loop',
  'payment': [{'f1': 375, 'f2': 2, 'f3': 12663},
   {'f1': 375, 'f2': 1, 'f3': 10640},
   {'f1': 375, 'f2': 2, 'f3': 10589},
   {'f1': 375, 'f2': 1, 'f3': 10274},
   {'f1': 375, 'f2': 1, 'f3': 1499},
   {'f1': 375, 'f2': 2, 'f3': 1404},
   {'f1': 375, 'f2': 1, 'f3': 2236},
   {'f1': 375, 'f2': 2, 'f3': 12122},
   {'f1': 375, 'f2': 2, 'f3': 8900},
   {'f1': 375, 'f2': 1, 'f3': 7856},
   {'f1': 375, 'f2': 2, 'f3': 5474},
   {'f1': 375, 'f2': 1, 'f3': 13836},
   {'f1': 375, 'f2': 2, 'f3': 11008},
   {'f1': 375, 'f2': 1, 'f3': 15004},
   {'f1': 375, 'f2': 1, 'f3': 10961},
   {'f1': 375, 'f2': 1, 'f3': 10859},
   {'f1': 375, 'f2': 2, 'f3': 4335},
   {'f1': 375, 'f2': 1, 'f3': 15505},
   {'f1': 375, 'f2': 1, 'f3': 3981},
   {'f1': 375, 'f2': 1, 'f3': 10672}]},
 {'customer_name': 'Adam Gooch',
  'email': 'adam.gooch@sakilacustomer.org',
  'address': '230 Urawa Dr

In [65]:
import json

filename = 'data.json'
file = open(filename, 'w')
json_string = json.dumps(data.json_agg[0])
file.write(json_string)
file.close()

### Analysing data with pandas

In [69]:
df = psql.read_sql('''
select first_name, country.country
from customer
left join address on address.address_id = customer.address_id
left join city on city.city_id = address.city_id
left join country on country.country_id = city.country_id;
''', conn)

df


,first_name,country
0,Jared,Indonesia
1,Mary,Japan
2,Patricia,United States
3,Linda,Greece
4,Barbara,Myanmar
...,...,...
594,Terrence,China
595,Enrique,Greece
596,Freddie,Peru
597,Wade,Switzerland


In [76]:
egypt = df[df['country'] == 'Egypt']
kuwait = df[df['country'] == 'Kuwait']
india = df[df['country'] == 'India']


In [83]:
egypt.head()

,first_name,country
119,Sherry,Egypt
134,Emma,Egypt
233,Lillie,Egypt
423,Alfred,Egypt
491,Rick,Egypt


In [78]:
kuwait

,first_name,country
234,Claudia,Kuwait


In [82]:
india.head()

,first_name,country
12,Nancy,India
15,Helen,India
28,Cynthia,India
31,Brenda,India
32,Amy,India


In [81]:
print('{} Customers in Egypt, {} customers in Kuwait and {} customers in India'.format(egypt.shape[0], kuwait.shape[0], india.shape[0]))

6 Customers in Egypt, 1 customers in Kuwait and 60 customers in India
